# Kinematic generator
This notebook converts a Denavit Hartenberg table to its direct kinematic symbolic calculation and its jacobian

In [1]:
from cmath import pi
import numpy as np
import sympy
# from sympy.abc import x,y,z,a,d
from kinematicBuilder import *

# import sys
# sys.set_int_max_str_digits(0)

# Examples

## 3 DOF Cartesian 
A 3 DOF Cartesian robot with 3 prismatic joints from [The Ultimate Guide to Jacobian Matrices for Robotics](https://automaticaddison.com/the-ultimate-guide-to-jacobian-matrices-for-robotics/)

In [2]:
T1 = DenavitRow(pi/2,   0, 0, pi/2,  Joint(sympy.Symbol('q_1'),JointType.PRISMATIC))
T2 = DenavitRow(pi/2,   0, 0, -pi/2, Joint(sympy.Symbol('q_2'),JointType.PRISMATIC))
T3 = DenavitRow(0,      0, 0, 0,     Joint(sympy.Symbol('q_3'),JointType.PRISMATIC))

T_cartesian = DenavitDK((T1,T2,T3),"Cartesian")

In [3]:
T_cartesian.inversePositionEval([0,0,0],[1,2,3])

0.0
Position solution with error 0.0 with 999 iterations left


array([ 3.,  1., -2.])

In [4]:
# T_cartesian.genURDF()

In [5]:
cartesian = T_cartesian.jacobian * T_cartesian.jointsSym
cartesian

Matrix([
[ q_2],
[-q_3],
[ q_1],
[   0],
[   0],
[   0]])

In [6]:
joint_values = (0, 0, 0)
T_cartesian.eval(joint_values)

array([[ 0, -1,  0,  0],
       [ 0,  0, -1,  0],
       [ 1,  0,  0,  0],
       [ 0,  0,  0,  1]])

## 3 DOF Articulated Robot
A 3 DOF Articulated robot with 3 rotational joints from [The Ultimate Guide to Jacobian Matrices for Robotics](https://automaticaddison.com/the-ultimate-guide-to-jacobian-matrices-for-robotics/)

In [7]:
T1 = DenavitRow(0,  1.5, 0,   pi/2, Joint(sympy.Symbol('q_1'),JointType.ROTATIONAL))
T2 = DenavitRow(0,  0,  1,  0,    Joint(sympy.Symbol('q_2'),JointType.ROTATIONAL))
T3 = DenavitRow(0,  0,  0.5,  0,    Joint(sympy.Symbol('q_3'),JointType.ROTATIONAL))

T_articular3 = DenavitDK((T1,T2,T3))

In [8]:
joint_values = (0, 0, 0)
T_articular3.eval(joint_values)

array([[ 1. , -0. ,  0. ,  1.5],
       [ 0. , -0. , -1. ,  0. ],
       [ 0. ,  1. ,  0. ,  1.5],
       [ 0. ,  0. ,  0. ,  1. ]])

## 6 DOF articular
A 6 DOF articular robot that follows the following scheme: 
<!-- ![axes](axes_diagram.png) -->
<img src="axes_diagram.png" alt="axes_diagram" width="400"/>

In [9]:
L = [0.2, 1, 1, 0.2]
T1 = DenavitRow(0,      L[0],   0,      pi/2,   Joint(sympy.Symbol('q_1'),JointType.ROTATIONAL,upper_limit=pi/2,lower_limit=-pi/2))
T2 = DenavitRow(pi/2,   0,      L[1],   0,      Joint(sympy.Symbol('q_2'),JointType.ROTATIONAL))
T3 = DenavitRow(pi,     0,      0,      pi/2,   Joint(sympy.Symbol('q_3'),JointType.ROTATIONAL))
# T_ = DenavitRow(pi/2,   0.02,       0,      pi/2)
T4 = DenavitRow(pi,     L[2],   0,      pi/2,   Joint(sympy.Symbol('q_4'),JointType.ROTATIONAL))
T5 = DenavitRow(pi,     0,      0,      pi/2,   Joint(sympy.Symbol('q_5'),JointType.ROTATIONAL))
T6 = DenavitRow(0,      L[3],   0,      0,      Joint(sympy.Symbol('q_6'),JointType.ROTATIONAL))

T_articular6 = DenavitDK((T1,T2,T3,T4,T5,T6))
T_articular6.directTransformSym

Matrix([
[(((-sin(q_2 + 884279719003555/562949953421312)*sin(q_3 + 884279719003555/281474976710656)*cos(q_1) + cos(q_1)*cos(q_2 + 884279719003555/562949953421312)*cos(q_3 + 884279719003555/281474976710656))*cos(q_4 + 884279719003555/281474976710656) + sin(q_1)*sin(q_4 + 884279719003555/281474976710656))*cos(q_5 + 884279719003555/281474976710656) + (sin(q_2 + 884279719003555/562949953421312)*cos(q_1)*cos(q_3 + 884279719003555/281474976710656) + sin(q_3 + 884279719003555/281474976710656)*cos(q_1)*cos(q_2 + 884279719003555/562949953421312))*sin(q_5 + 884279719003555/281474976710656))*cos(q_6) + ((-sin(q_2 + 884279719003555/562949953421312)*sin(q_3 + 884279719003555/281474976710656)*cos(q_1) + cos(q_1)*cos(q_2 + 884279719003555/562949953421312)*cos(q_3 + 884279719003555/281474976710656))*sin(q_4 + 884279719003555/281474976710656) - sin(q_1)*cos(q_4 + 884279719003555/281474976710656))*sin(q_6), -(((-sin(q_2 + 884279719003555/562949953421312)*sin(q_3 + 884279719003555/281474976710656)*cos(q_

In [10]:
# T_articular6.genURDF()
# T_articular6.genCCode()

In [11]:
pose = T_articular6.eval((1,1,1,1,1,1))
pose

array([[ 0.21667169, -0.26112258,  0.94067443, -0.04166873],
       [-0.95867353, -0.23890322,  0.1545002 , -0.32699789],
       [ 0.18438665, -0.93527549, -0.30209487, -0.2294141 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [12]:
T_articular6.inverseEval([0,0,0,0,0,0],pose)

0.5303672509456662
0.133090897045223
0.19805321952668054
0.058096235857168416
0.017616481038508602
0.0009623456241789342
1.7876297332903563e-06
Position solution with error 1.7876297332903563e-06 with 993 iterations left
18.121146966525668
2.1446655990862586
0.25126644087060407
0.0038410541378007565
8.995987686601186e-07
Solution with error 8.995987686601186e-07 with 995 iterations left


[4.141116140758804,
 5.282429591454324,
 2.140993919695244,
 4.141514676448182,
 0.9993505753533116,
 0.9989170096726199]

## 6 DOF robot with transformations in between

In [13]:
Lv = 50
T1 = DenavitRow(0,      0,          0,     -pi/2,   Joint(sympy.Symbol('q_1'),JointType.ROTATIONAL))
T2 = DenavitRow(0,      0,          Lv,     0,      Joint(sympy.Symbol('q_2'),JointType.ROTATIONAL))
T3 = DenavitRow(0,      -0.0425,    Lv,     pi/2,   Joint(sympy.Symbol('q_3'),JointType.ROTATIONAL))
T_ = DenavitRow(pi/2,   0.02,       0,      pi/2)
T4 = DenavitRow(pi/2,   0,          0,      pi/2,   Joint(sympy.Symbol('q_4'),JointType.ROTATIONAL))
T5 = DenavitRow(pi/2,   0,          0,      pi/2,   Joint(sympy.Symbol('q_5'),JointType.ROTATIONAL))
T6 = DenavitRow(0,      0,          0,      0,      Joint(sympy.Symbol('q_6'),JointType.ROTATIONAL))
TCP = DenavitRow(0,     0,          10,     0)

T_walter = DenavitDK((T1,T2,T3,T_,T4,T5,T6,TCP),"Walter")

In [14]:
T_walter.genURDF()
# T_walter.genCCode()

In [15]:
pose = T_walter.eval((1,1,1,1,1,1))
joints = T_walter.inverseEval([0,0,0,0,0,0],pose)
joints

833.7461170358483
214.28174789050013
59.02379496521931
9.98801344015011
0.2586509794326969
4.803857946481692e-05
Position solution with error 4.803857946481692e-05 with 994 iterations left
31501.757235671623
8698.474658487916
6547.043071951966
24126.727755450655
15316.94888786937
28168.71015006958
13309.198345216919
121.77392643168483
551.3232771784909
97.80846464287285
6.669588486516758
1.070252648677415
0.005541582723453081
1.7038291358261742e-07
Solution with error 1.7038291358261742e-07 with 986 iterations left


[1.0000399991201459,
 1.9996033169929692,
 5.283984112625572,
 2.4509040698799827,
 0.7760957613798043,
 5.585428943545787]

In [16]:
T_walter.eval(joints)

array([[-5.42953905e-01,  7.69269035e-01,  3.36788077e-01,
        -2.03021261e+00],
       [ 7.03943910e-01,  6.35611356e-01, -3.16956111e-01,
         1.22553776e+01],
       [-4.57890848e-01,  6.49873574e-02, -8.86629920e-01,
        -9.21256401e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [17]:
pose

array([[-5.42948980e-01,  7.69269352e-01,  3.36795292e-01,
        -2.02982706e+00],
       [ 7.03951467e-01,  6.35609326e-01, -3.16943396e-01,
         1.22555160e+01],
       [-4.57885069e-01,  6.50034461e-02, -8.86631725e-01,
        -9.21255942e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])